#**Artificial Neural Network Model for predicting iris**


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

1. **Loading Iris**

In [ ]:
iris = load_iris()

In [ ]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

2. **Converting data into x, y**

In [ ]:
X = iris['data']
y = iris['target']

3. **Standard Scaling of Features**


In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaled = scaler.fit_transform(X) # use to trnasform them into standar scaling

4. **Conversion of data into tensors**

In [ ]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32) # converting them into tensor format
y_tensor = torch.tensor(y, dtype=torch.long)

5. **Data Partitioning**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)

In [ ]:
train_ds = TensorDataset(X_train, y_train) # it is a requirement of the syntex for the model
test_ds = TensorDataset(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16) # batch size should not be very big nor very small i.e. 8, 16

In [ ]:
class AnnModel(nn.Module):
    def __init__(self):
        super(AnnModel, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(4, 16)
        self.Relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)

    # defining the flow of the data
    def forward(self, x):
        out = self.fc1(x)
        out = self.Relu(out)
        out = self.fc2(out)
        return out

In [ ]:
model = AnnModel()

In [ ]:
citerion = nn.CrossEntropyLoss() # calculates the difference
optimizer = optim.Adam(model.parameters(), lr=0.01) # updates the weights and biases at given rate

In [ ]:
epochs = 50 # one complete cycle through the entire training dataset during training

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad() # optimizer changes weight and bias at given rate
        outputs = model(batch_x) # this line applies model on batch_x (features) and store output in variable 'output'
        loss = citerion(outputs, batch_y) # Forward Propogation
        loss.backward() # Backward Propogation
        optimizer.step() # changes the weights and bias according to learning
        # by this point the modle is completed and trained the next lines are for personal evaluation
        total_loss +=loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss}")

Epoch: 0, Loss: 5.91192501783371
Epoch: 1, Loss: 4.064800649881363
Epoch: 2, Loss: 3.1265911757946014
Epoch: 3, Loss: 2.504741534590721
Epoch: 4, Loss: 2.2248829901218414
Epoch: 5, Loss: 1.9495065212249756
Epoch: 6, Loss: 1.751310020685196
Epoch: 7, Loss: 1.6177548095583916
Epoch: 8, Loss: 1.3733552396297455
Epoch: 9, Loss: 1.242641806602478
Epoch: 10, Loss: 1.0873157531023026
Epoch: 11, Loss: 0.9614417664706707
Epoch: 12, Loss: 0.9030695594847202
Epoch: 13, Loss: 0.7670299671590328
Epoch: 14, Loss: 0.6917162481695414
Epoch: 15, Loss: 0.7281034253537655
Epoch: 16, Loss: 0.6139706559479237
Epoch: 17, Loss: 0.5991733632981777
Epoch: 18, Loss: 0.5647460594773293
Epoch: 19, Loss: 0.5138627737760544
Epoch: 20, Loss: 0.5344760231673717
Epoch: 21, Loss: 0.5306311175227165
Epoch: 22, Loss: 0.45052226539701223
Epoch: 23, Loss: 0.4552547112107277
Epoch: 24, Loss: 0.545802092179656
Epoch: 25, Loss: 0.46812252467498183
Epoch: 26, Loss: 0.42932199500501156
Epoch: 27, Loss: 0.4111561430618167
Epoch:

In [ ]:
with torch.no_grad(): # this lines tells: don't change anything the model is already trained
    lst = []
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x) # the maximum number of output is the most likely prediction
        _, prediction = torch.max(outputs, 1) # this function gives the position of maximum output i.e. 0, 1, 2
        print("Prediction:    ", prediction)
        print("Actual Results:", batch_y) # now we are comparing the model predictions with actual results
        lst.append(prediction) # a list containing all batches of predictions

Prediction:     tensor([0, 2, 2, 2, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 1, 2])
Actual Results: tensor([0, 2, 2, 2, 0, 1, 1, 2, 0, 1, 2, 0, 2, 2, 2, 2])
Prediction:     tensor([1, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 0])
Actual Results: tensor([1, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 0])


In [ ]:
lst

[tensor([0, 2, 2, 2, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 1, 2]),
 tensor([1, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 0])]

In [ ]:
# now we are changing the predictions with actual names at given positions by manually looking at them
for i in lst[0]:
    if i.item() == 0:
        print("Setosa")
    elif i.item() == 1:
        print("Versicolor")
    else:
        print("Virginica")

Setosa
Virginica
Virginica
Virginica
Setosa
Versicolor
Versicolor
Virginica
Setosa
Virginica
Virginica
Setosa
Virginica
Virginica
Versicolor
Virginica


In [ ]:
iris['target_names'] # this line shows the names of outputs and their positions that we used in above code

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

# **Analysis & Conclusion**
Delving into the realm of artificial intelligence, this analysis focuses on an Artificial Neural Network (ANN) meticulously crafted for the classification of Iris species.

The process commenced with the acquisition of the Iris dataset, a crucial step before segregating the features (X) and the target variable (y). A vital preprocessing phase involved standardizing the features using `StandardScaler`, a technique employed to optimize the training process.

The standardized data was then transformed into PyTorch tensors and strategically divided into training and testing sets. The use of `TensorDataset` and `DataLoader` streamlined the process of handling data in batches during training.

An ANN model, characterized by a single hidden layer incorporating `nn.Linear` and `nn.ReLU`, and a three-node output layer, was subsequently defined. The training employed `CrossEntropyLoss` as the loss function and the Adam optimizer with a learning rate of 0.01.

Training spanned 50 epochs, during which a consistent decrease in loss was observed, signifying the model's successful learning progression.

The model's efficacy was ultimately assessed on the test set. Predictions were generated and compared against the actual values. While a precise quantitative accuracy was not calculated, the comparison offered valuable insights into the model's predictive prowess, with numerical predictions seamlessly mapped to their corresponding species names via `target_names`.